In [18]:
# from keras.layers import Input,Embedding,LSTM,Dense
# from keras.models import Model

# model = Sequential()

# model.add(Dense(64,input_dim=100))
# model.add(Activation('relu'))
# model.add(Dense(32))
# model.add(Activation('softmax'))
# model.compile(optimizer='sgd',loss='categorical_crossentropy')
# # model,fit()

############################# 生成GAN 模拟随机正弦曲线 #################################
%matplotlib
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Dense,Activation,Input,Reshape
from keras.layers import Conv1D,Conv2D
from keras.layers import Flatten,Dropout,Permute
from keras.optimizers import SGD,Adam,RMSprop

# from tqdm import tqdm_notebook as tqdm

num = 5000

def sample_data(n_sample= num, x_var= np.arange(0,3,0.05),
               max_offset=500, mul_range=[1,2]):
    vectors = []
    for i in range (n_sample):
        offset = np.random.random() 
        mul = mul_range[0] + np.random.random()*(mul_range[1]-
                                                mul_range[0])
        vectors.append(np.sin(offset + x_var * mul)/2 + 3)
        
    return np.array(vectors)

def get_generator(G_in, dense_dim=200, out_dim=80, lr=1e-3):
    x = Dense(dense_dim)(G_in)
    x = Activation('relu')(x)
    G_out = Dense(out_dim, activation='relu')(x)
    G = Model(G_in,G_out)
    opt = SGD(lr=lr)
    
    G.compile(loss='binary_crossentropy',optimizers=opt)
    
    return G,G_out

def get_discriminative(D_in,lr=1e-3, drate=.25,n_channels=50,
                       conv_sz=5,leak=.2):
    
    x = Reshape((-1,1))(D_in)
    x = Conv1D(n_channels, conv_sz, activation='relu')(x)
    x = Dropout(drate)
    x = Flatten()(x)
    x = Dense(n_channels)(x)
    D_out = Dense(2, activation='sigmoid')(x)
    D = Model(D_in, D_out)
    dopt = Adam(lr=lr)
    D.compile(loss='binary_crossentropy', optimizers=dopt)
    
    return D, D_out 

def set_trainability(model, trainable=False):
    model.trainable = trainable
    for layer in model.layers:
        layer.trainable = trainable
        
        
def make_GAN(GAN_in, G, D):
    set_trainability(D, False)
    x = G(GAN_in)
    x = G

Using matplotlib backend: Qt5Agg
